In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('pokemon.csv')
data.head()

,number,name,type1,type2,total,hp,attack,defense,sp_attack,sp_defense,speed,generation,legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,3,Gigantamax Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False


In [ ]:
data.isna().sum() # to check how many nan are there in each columns

,0
number,0
name,0
type1,0
type2,498
total,0
hp,0
attack,0
defense,0
sp_attack,0
sp_defense,0


In [ ]:
data  = data.drop(['type2','name','number'],axis=1)
data.isna().sum()

,0
type1,0
total,0
hp,0
attack,0
defense,0
sp_attack,0
sp_defense,0
speed,0
generation,0
legendary,0


In [ ]:
data['legendary'] = data['legendary'].astype(int)

In [ ]:
numeric_columns = data.drop(('type1') , axis=1)

In [ ]:
def oneHotEncoder(df , column , prefix ):

  df= df.copy()

  #now convert the dummy column which will contain the column with CATEGORICAL data and convert them into boolena data in serperate.....
  dummies = pd.get_dummies(df[column], prefix=prefix) # now dummies is a seprate dataframe which contains every category labels of column data
  dummies = dummies.astype(int) # convert them into 0 and 1

  df= pd.concat([df, dummies] ,axis=1) ## axis 1 means do with column and 0 means rows

  df = df.drop(column , axis=1)

  return df



In [ ]:
data = oneHotEncoder(data , 'type1' , 't') # we will replace class of type1 with their individual column and the pokemin with them will have 1 and 0 for other



In [ ]:
scaler = StandardScaler() # normalize data

X = data.drop(('legendary'), axis=1)
y = data['legendary']

In [ ]:
X = scaler.fit_transform(X)

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((857, 28), (215, 28), (857,), (215,))

In [ ]:
tf.random.set_seed(42)

Inputs  = tf.keras.Input(shape=(28,)) # 28 bcz its the number of features each of our sample has like we can also see that we have 857 samples and each has 28 features in it.....so we know we use features as input...it only defines shape of data it doesnt prepricess data , features  = columns

x = tf.keras.layers.Dense(70,activation="relu")(Inputs)
x = tf.keras.layers.Dense(35,activation="relu")(x)
outputs = tf.keras.layers.Dense(1,activation="sigmoid")(x) # it predicts the lengendary part as the output

# Breaking It Down
# First Dense Layer (x = Dense(70, activation="relu")(inputs))

# Input: 26 features (from tf.keras.Input(shape=(26,))).
# Output: 70 neurons, each applying ReLU activation.
# This extracts features from the input data.
# Second Dense Layer (x = Dense(70, activation="relu")(x))

# Input: The output from the first Dense layer (70 values).
# Output: Another 70 neurons, applying ReLU.
# This adds complexity and allows the network to learn more abstract features.

# so first hidden layer takes input from Inputs layer and then produces an output....then that output goes as input to next hidden layer and so on...


model = tf.keras.Model(inputs=Inputs,outputs=outputs) # defines input tensor and output tensor

model.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics = ['accuracy']
)

batch_size = 32
epochs= 100

history = model.fit(X_train ,
                    y_train,
                    validation_split = 0.2,
                    batch_size  =batch_size,
                    epochs = epochs)



Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.7915 - loss: 0.4002 - val_accuracy: 0.9360 - val_loss: 0.1770
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9472 - loss: 0.1406 - val_accuracy: 0.9360 - val_loss: 0.1755
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9637 - loss: 0.0961 - val_accuracy: 0.9244 - val_loss: 0.1834
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9552 - loss: 0.1101 - val_accuracy: 0.9419 - val_loss: 0.1671
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9733 - loss: 0.0909 - val_accuracy: 0.9477 - val_loss: 0.1836
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9710 - loss: 0.0808 - val_accuracy: 0.9593 - val_loss: 0.2004
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9895 - loss: 0.0554 - val_accuracy: 0.9593 - val_loss: 0.2307
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9901 - loss: 0.0454 - val_accuracy: 0.9535 - 

batch_size=32

This controls how many samples the model will process before updating the weights during training.
32 means the model will train on 32 samples at a time, and once it processes those, it will update the weights and continue to the next batch of 32 samples.

Why batch size matters:
A smaller batch size may make the training more "noisy" but can help the model generalize better.
A larger batch size provides more stable gradients but can be slower and risk overfitting.
32 is often a good starting point because it's a balance between training time and performance.

Why Use validation_split=0.2?
The validation data is used to monitor the model's performance during training, but it is not used for training. It helps you see how well the model generalizes to data it hasn't seen before.

validation_split=0.2 means that 20% of your training data will be set aside for validation, and the model will be trained on the remaining 80%.

No Need to Create a Separate Validation Set:

When you use validation_split, Keras will automatically split the data into training and validation sets for you.
You don't have to create a separate validation dataset beforehand—Keras will take 20% of the data from X_train and y_train for validation.

Important Note: The validation data comes from the training data but is not used in training—it's set aside for validation purposes.

When you use validation_split=0.2 in model.fit(), you're telling Keras to automatically reserve 20% of your training data for validation. So, if you have 1000 training samples:

Training data: 800 samples (80% of 1000)
Validation data: 200 samples (20% of 1000)
Keras uses the 80% to train the model, and after each epoch, it evaluates the model on the 20% validation data to check how well it generalizes.

If you have 100 samples and split them like this:
80 for training and 20 for testing

Training data (80 samples): This is the data your model will learn from.
Test data (20 samples): This is used for evaluating the model after training is complete.
validation_split=0.2

Validation data is taken from the training data (not from the entire dataset).
So, when you set validation_split=0.2, Keras will hold back 20% of the training data (i.e., 20% of 80 samples) as the validation data.
Therefore, 16 samples will be used for validation, and the model will be trained on the remaining 64 samples (80 - 16).


 [Input Layer]      - 26 features (Pokémon stats, type, etc.)

     ↓
 [Hidden Layer 1]   - 70 neurons (ReLU activation)

     ↓
 [Hidden Layer 2]   - 70 neurons (ReLU activation)

     ↓
 [Output Layer]     - 12 neurons (Softmax activation, if classifying 12 Pokémon types)




In [ ]:
# its like

# model = Sequential([
#     Dense(70, activation="relu", input_shape=(26,)),  # First hidden layer (input layer implicitly defined)
#     Dense(70, activation="relu"),  # Second hidden layer
#     Dense(12, activation="softmax")  # Output layer (assuming 12 classes for Pokémon Type1 classification)
# ])
# Yes, exactly! In the Sequential API, you don't need to explicitly mention that one hidden layer’s output goes into the next. The Sequential model automatically connects layers in order.

In [ ]:
!pip install plotly
import plotly.express as px

fig_loss= px.line(
    history.history,
    y=['loss','val_loss'],
    labels={
        'index':'epoch',
        'value':'loss'
    },
    title='loss over time'
)

fig_loss.show()

In [ ]:
np.argmin(history.history['val_loss'])

3

In [ ]:
model_predict = model.predict(X_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


In [ ]:
np.stack((model_predict >= 0.5).astype(int) , axis=1) # make them as column
# and if model_predict >= 0.5 classify it as true else false , its like the threshold then astype converts them into 0 and 1 ,

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

In [ ]:
predictions = (np.hstack((model_predict >= 0.5).astype(int)) == y_test).astype(int)
predictions # 1 means prediction was correct while 0 means prediction wasnt correct

,legendary
992,1
746,1
1066,0
425,1
490,1
...,...
634,1
486,1
298,1
582,1


In [ ]:
predictions.sum() , len(y_test)

(200, 215)

In [ ]:
# so 200 out of 215 were correct

accuracy= 1-15/215
accuracy

0.9302325581395349